In [120]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

from IPython.display import display, Math
from scipy import optimize
from math import exp, cos, sqrt, pi, sin, sqrt

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset

mypalette = sns.color_palette(['#51ffda']) # https://www.hsluv.org/
sns.set_palette(mypalette)

## Constants to be used in the notebook

In [121]:
# The depth of the aquifer in meters. Values are around 30.
depth_aquifer = float(input("Aquifer temperature depth: "))

Aquifer temperature depth:  30


## Datasets reading

In [122]:
ospitaletto2019 = OspitalettoDataset()
noaa2010Dataset = NOAA2010Dataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(ospitaletto2019.load_all_data())
AVAILABLE_DATASETS.update(noaa2010Dataset.load_data())
AVAILABLE_DATASETS.keys()

dict_keys(['ospitaletto', 'miami_florida', 'fresno_california', 'olympia_washington', 'rochester_newyork'])

In [182]:
dataset_to_work = "rochester_newyork"
dataset_to_work

'rochester_newyork'

In [183]:
dataset = AVAILABLE_DATASETS[dataset_to_work]
dataset

,station_id,station_name,air_temp
timestamp,,,
2010-01-01 01:00:00,USW00014768,"ROCHESTER GREATER INTERNATIONAL, NY US",-2.9
2010-01-01 02:00:00,USW00014768,"ROCHESTER GREATER INTERNATIONAL, NY US",-3.1
2010-01-01 03:00:00,USW00014768,"ROCHESTER GREATER INTERNATIONAL, NY US",-3.2
2010-01-01 04:00:00,USW00014768,"ROCHESTER GREATER INTERNATIONAL, NY US",-3.3
2010-01-01 05:00:00,USW00014768,"ROCHESTER GREATER INTERNATIONAL, NY US",-3.3
...,...,...,...
2010-12-31 19:00:00,USW00014768,"ROCHESTER GREATER INTERNATIONAL, NY US",-1.9
2010-12-31 20:00:00,USW00014768,"ROCHESTER GREATER INTERNATIONAL, NY US",-2.2
2010-12-31 21:00:00,USW00014768,"ROCHESTER GREATER INTERNATIONAL, NY US",-2.3


# Definition of the tentative values of Tamb_avg, DT_year, d_shift_min and d_shift_max

In [184]:
#Este codigo asume que el dataset tiene un index de tiempo.
T_hourly_avg = dataset['air_temp'].resample('H').mean().to_frame() # DataFrame de valores promedios en escala de tiempo horaria
T_hourly_avg["dayofyear"] = T_hourly_avg.index.dayofyear
T_hourly_avg["month"] = T_hourly_avg.index.month
T_hourly_avg["hourofyear"] = (T_hourly_avg.index.dayofyear - 1) * 24 + (T_hourly_avg.index.hour + 1)

T_daily_avg = dataset["air_temp"].resample('D').mean().to_frame() # DataFrame de valores promedios en escala de tiempo diaria
T_daily_avg["dayofyear"] = T_daily_avg.index.dayofyear
T_daily_avg["month"] = T_daily_avg.index.month
T_daily_avg["hourofyear"] = (T_daily_avg.index.dayofyear - 1) * 24 + (T_daily_avg.index.hour + 1)

T_monthly_avg = dataset["air_temp"].resample('M').mean().to_frame() # DataFrame de valores promedios en escala de tiempo mensual
T_monthly_avg["dayofyear"] = T_monthly_avg.index.dayofyear
T_monthly_avg["month"] = T_monthly_avg.index.month
T_monthly_avg["hourofyear"] = (T_monthly_avg.index.dayofyear - 1) * 24 + (T_monthly_avg.index.hour + 1)

T_ave_h = T_daily_avg.air_temp.mean() # Average dataset temperature on a daily basis
T_max = T_daily_avg.air_temp.max() # Max dataset temperature on a daily basis
DT_y = T_max - T_ave_h # Temperature swing Tmax - Tmean

d_shift_min = T_daily_avg.air_temp.idxmin(axis=0).dayofyear #Day of the min T
d_shift_max = T_daily_avg.air_temp.idxmax(axis=0).dayofyear #Day of the max T

t = T_daily_avg.dayofyear

num_hours = T_hourly_avg.shape[0] # Number of hours in a year =8737 hrs
h = pd.Series(range(1, num_hours + 1)) # A range equivalent to the hours of the year from 1 to 8761

print(f"* {T_hourly_avg=}\n"
      f"* {T_daily_avg=}\n"
      f"* {T_monthly_avg=}\n"
      f"* {T_ave_h=}\n"
      f"* {DT_y=}\n"
      f"* {d_shift_min=}\n"
      f"* {T_daily_avg.air_temp.min()=}\n"
      f"* {d_shift_max=}\n"
      f"* {T_daily_avg.air_temp.max()=}\n"
      f"* {t=}\n"
      f"* {num_hours=}\n"
      f"* {h=}")

* T_hourly_avg=                     air_temp  dayofyear  month  hourofyear
timestamp                                                  
2010-01-01 01:00:00      -2.9          1      1           2
2010-01-01 02:00:00      -3.1          1      1           3
2010-01-01 03:00:00      -3.2          1      1           4
2010-01-01 04:00:00      -3.3          1      1           5
2010-01-01 05:00:00      -3.3          1      1           6
...                       ...        ...    ...         ...
2010-12-31 19:00:00      -1.9        365     12        8756
2010-12-31 20:00:00      -2.2        365     12        8757
2010-12-31 21:00:00      -2.3        365     12        8758
2010-12-31 22:00:00      -2.6        365     12        8759
2010-12-31 23:00:00      -2.7        365     12        8760

[8759 rows x 4 columns]
* T_daily_avg=            air_temp  dayofyear  month  hourofyear
timestamp                                         
2010-01-01 -2.291304          1      1           1
2010-01-02 -2

In [185]:
# Verify integrity of the data
hours_of_year = T_hourly_avg.hourofyear # (T_hourly_avg.index.dayofyear - 1) * 24 + (T_hourly_avg.index.hour + 1)
hours_not_in_data = set(range(1, num_hours + 1)) - set(hours_of_year)

days_of_year = T_hourly_avg.dayofyear
days_not_in_data = set(range(1, 365 + 1)) - set(days_of_year)

months_of_year = T_hourly_avg.month
months_not_in_data = set(range(1, 12 + 1)) - set(months_of_year)

assert len(hours_not_in_data) == 0, f"ALERT - The dataset does not contain the following hours: {hours_not_in_data}"
assert len(days_not_in_data) == 0, f"ALERT - The dataset does not contain the following days: {days_not_in_data}"
assert len(months_not_in_data) == 0, f"ALERT - The dataset does not contain the following months: {months_not_in_data}"

AssertionError: ALERT - The dataset does not contain the following hours: {1}

# This code finds the optimal curve that better fits the raw data through the minimum square method 

For more info visit the [scipy.optimize.curve_fit docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit)

In [186]:
def func(x, disp, amp, phi):
    t0 = len(h) # hourly resolution if x is in hours
    omega = 2 * pi / t0
    return disp + amp * np.cos(x * omega - phi) 

x_data = pd.Series(range(0,len(T_hourly_avg)))
y_data = T_hourly_avg.air_temp
params, params_covariance = optimize.curve_fit(func, x_data, y_data, p0=[T_ave_h, DT_y, d_shift_max * 24])
print(f"* {params=}\n"
      f"* {params_covariance=}")

* params=array([   9.18765842,   12.77655797, 4822.71071655])
* params_covariance=array([[6.92078579e-04, 1.87375366e-13, 2.54759363e-15],
       [1.87375366e-13, 1.38415716e-03, 3.89272878e-09],
       [2.54759363e-15, 3.89272878e-09, 8.47925900e-06]])


In [187]:
T_ave_fit = params[0]
DT_y_fit = params[1]
phi = params[2] #Corresponds to day 202

#'displacement, amplitude, and phase of the signal
print('Fitted parameters:')
display(Math(f"dist={T_ave_fit:.2f}, amp={DT_y_fit:.2f}, \\phi={phi:.2f}"))

print('Original parameters:')
display(Math(f"dist={T_ave_h:.2f}, amp={DT_y:.2f}, \\phi={d_shift_max*24:.2f}"))

Fitted parameters:


<IPython.core.display.Math object>

Original parameters:


<IPython.core.display.Math object>

# The new ambient temperature curve uses the fitted parameters as input

In [188]:
def fitting_curve(hourofyear, T_ave_fit, DT_y_fit, T, phi, pi):
    omega = 2 * pi / T
    return T_ave_fit + DT_y_fit * np.cos(hourofyear * omega - phi) 

In [189]:
T = T_hourly_avg.hourofyear.shape[0]
T_hourly_avg["air_temp_fit"] = T_hourly_avg.hourofyear.apply(fitting_curve, args=(T_ave_fit, DT_y_fit, T, phi, pi))

d_shift_min = T_hourly_avg.air_temp_fit.idxmin(axis=0)
d_shift_max = T_hourly_avg.air_temp_fit.idxmax(axis=0)

dd_min = T_hourly_avg[T_hourly_avg.index == d_shift_min].dayofyear.item() # d_shift_min / 24
dd_max = T_hourly_avg[T_hourly_avg.index == d_shift_max].dayofyear.item() # d_shift_max / 24

print(f'* {T=}\n'
      f'* {T_hourly_avg=}\n'
      f'* d_min (h): {d_shift_min}\n'
      f'* day_number: {dd_min}\n'
      f'* d_max (h): {d_shift_max}\n'
      f'* day_number: {dd_max}')

* T=8759
* T_hourly_avg=                     air_temp  dayofyear  month  hourofyear  air_temp_fit
timestamp                                                                
2010-01-01 01:00:00      -2.9          1      1           2     -2.749240
2010-01-01 02:00:00      -3.1          1      1           3     -2.752504
2010-01-01 03:00:00      -3.2          1      1           4     -2.755763
2010-01-01 04:00:00      -3.3          1      1           5     -2.759015
2010-01-01 05:00:00      -3.3          1      1           6     -2.762261
...                       ...        ...    ...         ...           ...
2010-12-31 19:00:00      -1.9        365     12        8756     -2.732825
2010-12-31 20:00:00      -2.2        365     12        8757     -2.736120
2010-12-31 21:00:00      -2.3        365     12        8758     -2.739409
2010-12-31 22:00:00      -2.6        365     12        8759     -2.742692
2010-12-31 23:00:00      -2.7        365     12        8760     -2.745969

[8759 rows x 

## Ground temperature as a function of the ambient temperature fitting curve

See for Banks [Measurements of Ground Temperature at Various Depths](https://www.researchgate.net/publication/30500372_Measurements_of_Ground_Temperature_at_Various_Depths?enrichId=rgreq-e2031a024b742c0018bb428dca3100f5-XXX&enrichSource=Y292ZXJQYWdlOzMwNTAwMzcyO0FTOjEwMTI0NTIzNDcxMjU4M0AxNDAxMTUwMTUzNjYy&el=1_x_2&_esc=publicationCoverPdf) and for Kusuda [Earth Temperatures and Thermal Diffusivity at Selected Stations in the United States](https://nvlpubs.nist.gov/nistpubs/Legacy/RPT/nbsreport8972.pdf)

In [190]:
# The purpose of having two equations is to show that they both work using the same input data
zz = 1 # Depth [m]
alpha = 0.06048 # Ground thermal diffusivity, Banks [m^2/day]
alpha_sec = 7e-7 # Ground thermal diffusivity, Banks [m^2/s]
t_sec = 365 * 24 * 3600 # Number of seconds in a year.

t_0 = T_daily_avg.shape[0] # Number of days in a year =365 days

Tg_und = T_ave_fit # Undisturbed ground temperature
DT_y = abs(DT_y_fit)

print(f"* {zz=}\n"
      f"* {alpha=}\n"
      f"* {alpha_sec=}\n"
      f"* {t_sec=}\n"
      f"* {t_0=}\n"
      f"* {Tg_und=}\n"
      f"* {DT_y=}\n"
      f"* {dd_min=}\n"
      f"* {dd_max=}")

* zz=1
* alpha=0.06048
* alpha_sec=7e-07
* t_sec=31536000
* t_0=365
* Tg_und=9.187658415231072
* DT_y=12.776557965468042
* dd_min=22
* dd_max=204


In [191]:
def ground_temperature_hour(t: int):
    #t is time in hours, but the calculation is done is seconds
    T_banks= Tg_und+DT_y*exp(-zz*sqrt(pi/(alpha_sec*t_sec)))*cos(2*pi/t_sec*(t-dd_max*24)*3600-zz*sqrt(pi/(alpha_sec*t_sec))) #Banks + t_shift ->Marco's
    
    T_kusuda = Tg_und-DT_y*exp(-zz*sqrt(pi/(alpha_sec*t_sec)))*cos(2*pi/t_sec*((t-dd_min*24)*3600-zz/2*sqrt(t_sec/(pi*alpha_sec)))) #Kusuda
    return  (pd.Series({'T_banks': T_banks,'T_kusuda': T_kusuda}))

def ground_temperature_day(t: int):
    # t corresponds to the number of day from 1 to 365
    T_banks = Tg_und + DT_y * exp(-zz * sqrt(pi / (alpha * t_0))) * cos(2 * pi / t_0 * (t - dd_max) - zz * sqrt(pi / (alpha * t_0))) #Banks + t_shift 
    T_kusuda = Tg_und - DT_y * exp(-zz * sqrt(pi / (alpha * t_0))) * cos(2 * pi / t_0 * (t - dd_min - zz / 2 * sqrt(t_0 / (pi * alpha)))) #Kusuda
    return  (pd.Series({'T_banks': T_banks,'T_kusuda': T_kusuda}))

def ground_temperature_month(month: int):
    #t corresponds to the number of month from 1 to 12
    T_banks= Tg_und+DT_y*exp(-zz*sqrt(pi/(alpha*t_0)))*cos(2*pi/t_0*(15+(month-1)*30-dd_max)-zz*sqrt(pi/(alpha*t_0))) #Banks + t_shift 
    
    T_kusuda = Tg_und-DT_y*exp(-zz*sqrt(pi/(alpha*t_0)))*cos(2*pi/t_0*(15+(month-1)*30-dd_min-zz/2*sqrt(t_0/(pi*alpha)))) #Kusuda ->Selva's
    return  (pd.Series({'T_banks': T_banks,'T_kusuda': T_kusuda}))

In [192]:
tbanks_and_tkusuda = T_hourly_avg.hourofyear.apply(ground_temperature_hour) # T_daily_avg.hourofyear.apply(ground_temperature_hour)

T_hourly_avg["ground_temp"] = tbanks_and_tkusuda["T_banks"]
# T_hourly_avg[["air_temp", "ground_temp"]].plot(colormap='cool', figsize=(12,6))

T_hourly_avg

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp
timestamp,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587
...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705


## Aquifer temperature as a function of the ambient temperature fitting curve

We use the same equations as for ground temp

In [193]:
zz = depth_aquifer

print(f"* {zz=}\n"
      f"* {alpha=}\n"
      f"* {alpha_sec=}\n"
      f"* {t_sec=}\n"
      f"* {t_0=}\n"
      f"* {Tg_und=}\n"
      f"* {DT_y=}\n"
      f"* {dd_min=}\n"
      f"* {dd_max=}")

* zz=30.0
* alpha=0.06048
* alpha_sec=7e-07
* t_sec=31536000
* t_0=365
* Tg_und=9.187658415231072
* DT_y=12.776557965468042
* dd_min=22
* dd_max=204


In [194]:
tbanks_and_tkusuda = T_hourly_avg.hourofyear.apply(ground_temperature_hour) # T_daily_avg.hourofyear.apply(ground_temperature_hour)

T_hourly_avg["aquifer_temp"] = tbanks_and_tkusuda["T_banks"]
# T_hourly_avg[["air_temp", "aquifer_temp"]].plot(colormap='cool', figsize=(12,6))

T_hourly_avg

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp
timestamp,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560
...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559


## Save CSV version of the dataset

In [195]:
AVAILABLE_DATASET_PATHS = {"ospitaletto": ospitaletto2019.processed_dataset_path,
                           "miami_florida": noaa2010Dataset.processed_miami_fl_dataset_path, 
                           "fresno_california": noaa2010Dataset.processed_fresno_ca_dataset_path, 
                           "olympia_washington": noaa2010Dataset.processed_olympia_wa_dataset_path,
                           "rochester_newyork": noaa2010Dataset.processed_rochester_ny_dataset_path}

T_hourly_avg.to_csv(path_or_buf=AVAILABLE_DATASET_PATHS[dataset_to_work])